In [13]:
from langchain_community.document_loaders import PyPDFLoader

# Specify the path to your PDF file
pdf_path = "./documents/Fahim-CV.pdf"

# Create a PDF loader
loader = PyPDFLoader(pdf_path)

# Load the document into LangChain
documents = loader.load()

# Check the loaded content
for doc in documents:
    print(doc.page_content)

Mohammad Zobair Hossain Fahim
zobairf03@gmail.com
 
01840219368
 
Chittagong,Bangladesh
 
Github
 
Portfolio
 
PROFILE
Recent computer science graduate with a year of experience at The WOS Group GmbH in their major project
4takeaway. Initially served as a manual tester and was promoted to Junior Software Developer. A detail-
oriented and adaptable individual with a strong work ethic, committed to contributing technical skills and
problem-solving abilities to dynamic software development teams. Eager to embark on a career in software
development, combining academic excellence with practical experience and a passion for continuous
learning.My goal is to contribute cutting-edge solutions to challenging projects in collaborative environments,
always staying at the forefront of web development trends.
PROFESSIONAL EXPERIENCE
Junior Software Developer
The WOS Group GmbH(Remote)
•Implemented  & fixed some major component for their app and website project 
using Flutter and NextJS framework.
0

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(documents)

In [3]:
print(doc_splits)

[Document(metadata={'source': './tahsinCVedit.pdf', 'page': 0}, page_content='TAHSIN AHMED\nMAJUMDER\nASPIRE TO BE A SOFTWARE\nENGINEER\nUndergraduate student in CSE at\nChittagong University of Engineering &\nTechnology who loves to solve problems\nwith C++ and keen to learn the real-\nworld technologies.\nAirline Ticketing System with OTP verification \n(CONTRIBUTOR AS MOHSINIAN)\nPROJECTS\nProblem solving in different online judges\nBlockchain & Web 3.0 technologies\nDevOps Engineering\nBackend Web Development\nINTERESTS\n01771706933\ntahsinbd.ahmed1@gmail.com\nEDUCATIONS\nChittagong University of Engineering &\nTechnology\nBSC IN COMPUTER SCIENCE &\nENGINEERING\nJANUARY 2018 - PRESENT\nPARTICIPATIONS\nGovt. Hazi Mohammad Mohsin College,\nChittagong\nHSC IN SCIENCE\nJULY, 2015 – APRIL, 2017 \nChittagong Collegiate School\nSSC IN SCIENCE\nJANUARY, 2009 – FEBRUARY 2015\nNASA Space Apps Challenge 2021'), Document(metadata={'source': './tahsinCVedit.pdf', 'page': 0}, page_content='Googl

In [15]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
embeddings = OllamaEmbeddings(model='mxbai-embed-large')

# Convert documents to vectors and index vectors
db = FAISS.from_documents(doc_splits, embeddings)

In [17]:
retriever = db.as_retriever(k=4)

In [18]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [19]:
# Initialize the LLM with Llama 3.1 model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [20]:
rag_chain = prompt | llm | StrOutputParser()

In [22]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [25]:
rag_application = RAGApplication(retriever, rag_chain)
question = "Why should not I hire Fahim?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Why should not I hire Fahim?
Answer: I don't know. The provided documents only contain Fahim's resume, education background, and work experience, but do not mention any reasons why he should or shouldn't be hired.
